In [1]:
from sklearn.datasets import fetch_20newsgroups
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]
fetch_subset = lambda subset: fetch_20newsgroups(
    subset=subset, categories=categories,
    shuffle=True, random_state=42,
    remove=('headers', 'footers', 'quotes'))
train = fetch_subset('train')
test = fetch_subset('test')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 4))
clf = SGDClassifier(n_jobs=-1)
pipeline = Pipeline([('vec', vec), ('clf', clf)])
pipeline.fit(train['data'], train['target'])

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(3, 4), norm='l2', preprocessor=None, smooth_idf=True,
...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))])

In [3]:
from eli5 import explain_weights, explain_prediction
from eli5 import format_as_html, format_as_text, format_html_styles

print(format_as_text(explain_weights(clf, vec, target_names=train['target_names'])))

/Users/kostia/shub/memex/eli5/venv/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Explained as: linear model

Features with largest coefficients per class.
Caveats:
1. Be careful with features which are not
   independent - weights don't show their importance.
2. If scale of input features is different then scale of coefficients
   will also be different, making direct comparison between coefficient values
   incorrect.
3. Depending on regularization, rare features sometimes may have high
   coefficients; this doesn't mean they contribute much to the
   classification result for most examples.

y='alt.atheism' top features
--------------
  +2.834  heis
  +2.299  eis 
  +2.086  eist
  +2.003  ░ath
  +1.851  athe
  +1.810  thei
  +1.702  hei 
  +1.679  sla 
  +1.669  ░pos
  +1.586  nat 
  +1.526  mott
  +1.526  post
  +1.503  slam
  +1.466  stin
  +1.453  ath 
  +1.438  rna 
  +1.434  ░mad
  +1.420  obb 
       …  (20056 more positive features)
       …  (33424 more negative features)
  -1.410  ░ro 
  -1.477  pac 

y='comp.graphics' top features
--------------
  +2.15

In [4]:
from IPython.core.display import display, HTML
show_html = lambda html: display(HTML(html))
show_html_expl = lambda expl, **kwargs: show_html(format_as_html(expl, include_styles=False, **kwargs))
show_html(format_html_styles())

In [5]:
show_html_expl(explain_weights(clf, vec, target_names=train['target_names'], top=100))

In [6]:
show_html_expl(explain_prediction(clf, test['data'][7], vec, target_names=train['target_names'], top=50), force_weights=True)

In [7]:
show_html_expl(explain_prediction(clf, test['data'][1], vec, target_names=train['target_names']))

In [8]:
import numpy as np
for doc in test['data'][:10]:
    expl = explain_prediction(clf, doc, vec, target_names=train['target_names'])
    # haaack - leave only the winner
    max_class_idx = np.argmax([t.score for t in expl.targets])
    expl.targets = [expl.targets[max_class_idx]]
    show_html_expl(expl, force_weights=False)

Weight,Feature
+1.510,Highlighted in text (sum)
-0.984,<BIAS>


Weight,Feature
+2.740,Highlighted in text (sum)
-0.948,<BIAS>


Weight,Feature
+2.793,Highlighted in text (sum)
-0.948,<BIAS>


Weight,Feature
+1.880,Highlighted in text (sum)
-0.948,<BIAS>


Weight,Feature
+2.050,Highlighted in text (sum)
-0.948,<BIAS>


Weight,Feature
+1.688,Highlighted in text (sum)
-0.948,<BIAS>


Weight,Feature
+0.451,Highlighted in text (sum)
-0.948,<BIAS>


Weight,Feature
+2.786,Highlighted in text (sum)
-0.984,<BIAS>


Weight,Feature
+3.084,Highlighted in text (sum)
-0.999,<BIAS>


Weight,Feature
+1.237,Highlighted in text (sum)
-0.984,<BIAS>
